<a href="https://colab.research.google.com/github/iamA333/PowerBI-Dashboards/blob/main/YOLOv5_Classification_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# YOLOv5 Classification Tutorial

YOLOv5 supports classification tasks too. This is the official YOLOv5 classification notebook tutorial. YOLOv5 is maintained by [Ultralytics](https://github.com/ultralytics/yolov5).

This notebook covers:

*   Inference with out-of-the-box YOLOv5 classification on ImageNet
*  [Training YOLOv5 classification](https://blog.roboflow.com//train-YOLOv5-classification-custom-data) on custom data

*Looking for custom data? Explore over 66M community datasets on [Roboflow Universe](https://universe.roboflow.com).*

This notebook was created with Google Colab. [Click here](https://colab.research.google.com/drive/1FiSNz9f_nT8aFtDEU3iDAQKlPT8SCVni?usp=sharing) to run it.

# Setup

Pull in respective libraries to prepare the notebook environment.

In [1]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install

import torch
import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v7.0-245-g3d8f004 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 26.9/78.2 GB disk)


# 1. Infer on ImageNet

To demonstrate YOLOv5 classification, we'll leverage an already trained model. In this case, we'll download the ImageNet trained models pretrained on ImageNet using YOLOv5 Utils.

In [ ]:
from utils.downloads import attempt_download

p5 = ['n', 's', 'm', 'l', 'x']  # P5 models
cls = [f'{x}-cls' for x in p5]  # classification models

for x in cls:
    attempt_download(f'weights/yolov5{x}.pt')

Now, we can infer on an example image from the ImageNet dataset.

In [ ]:
#Download example image
import requests
image_url = "https://i.imgur.com/OczPfaz.jpg"
img_data = requests.get(image_url).content
with open('bananas.jpg', 'wb') as handler:
    handler.write(img_data)

In [ ]:
#Infer using classify/predict.py
!python classify/predict.py --weights ./weigths/yolov5s-cls.pt --source bananas.jpg

From the output, we can see the ImageNet trained model correctly predicts the class `banana` with `0.95` confidence.

## 2. (Optional) Validate

Use the `classify/val.py` script to run validation for the model. This will show us the model's performance on each class.

First, we need to download ImageNet.

In [ ]:
# # WARNING: takes ~20 minutes
# !bash data/scripts/get_imagenet.sh --val

In [ ]:
# # run the validation script
# !python classify/val.py --weights ./weigths/yolov5s-cls.pt --data ../datasets/imagenet

The output shows accuracy metrics for the ImageNet validation dataset including per class accuracy.

# 3. Train On Custom Data

To train on custom data, we need to prepare a dataset with custom labels.

To prepare custom data, we'll use [Roboflow](https://roboflow.com). Roboflow enables easy dataset prep with your team, including labeling, formatting into the right export format, deploying, and active learning with a `pip` package.

If you need custom data, there are over 66M open source images from the community on [Roboflow Universe](https://universe.roboflow.com).

(For more guidance, here's a detailed blog on [training YOLOv5 classification on custom data](https://blog.roboflow.com/train-YOLOv5-classification-custom-data).)


Create a free Roboflow account, upload your data, and label.

![](https://s4.gifyu.com/images/fruit-labeling.gif)

### Load Custom Dataset

Next, we'll export our dataset into the right directory structure for training YOLOv5 classification to load into this notebook. Select the `Export` button at the top of the version page, `Folder Structure` type, and `show download code`.

The ensures all our directories are in the right format:

```
dataset
├── train
│   ├── class-one
│   │   ├── IMG_123.jpg
│   └── class-two
│       ├── IMG_456.jpg
├── valid
│   ├── class-one
│   │   ├── IMG_789.jpg
│   └── class-two
│       ├── IMG_101.jpg
├── test
│   ├── class-one
│   │   ├── IMG_121.jpg
│   └── class-two
│       ├── IMG_341.jpg
```

![](https://i.imgur.com/BF9BNR8.gif)


Copy and paste that snippet into the cell below.

In [3]:
# Ensure we're in the right directory to download our custom dataset
import os
os.makedirs("../datasets/", exist_ok=True)
%cd ../datasets/

/datasets


In [4]:
# !pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="OLEusDLSXJg1Kbc171Xa")
project = rf.workspace("nmims-oawfg").project("sign-language-detectiom")
dataset = project.version(1).download("folder")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Sign-Language-Detectiom-1 in folder:: 100%|██████████| 26027/26027 [00:02<00:00, 10568.45it/s]


In [5]:
#Save the dataset name to the environment so we can use it in a system call later
dataset_name = dataset.location.split(os.sep)[-1]
os.environ["DATASET_NAME"] = dataset_name

### Train On Custom Data 🎉
Here, we use the DATASET_NAME environment variable to pass our dataset to the `--data` parameter.

Note: we're training for 100 epochs here. We're also starting training from the pretrained weights. Larger datasets will likely benefit from longer training.

In [6]:
%cd ../yolov5
!python /content/yolov5/classify/train.py --model yolov5s-cls.pt --data $DATASET_NAME --epochs 100 --img 128 --pretrained weights/yolov5s-cls.pt

[Errno 2] No such file or directory: '../yolov5'
/datasets
2023-11-23 10:41:45.786308: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-23 10:41:45.786378: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-23 10:41:45.786426: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
classify/train: model=yolov5s-cls.pt, data=Sign-Language-Detectiom-1, epochs=100, batch_size=64, imgsz=128, nosave=False, cache=None, device=, workers=8, project=../content/yolov5/runs/train-cls, name=exp, exist_ok=False, pretrained=weights/yolov5s-cls.pt, optimizer=Adam, lr0=0.001, decay=5e-05, label_

### Validate Your Custom Model

Repeat step 2 from above to test and validate your custom model.

In [7]:
!python /content/yolov5/classify/val.py --weights /content/yolov5/runs/train-cls/exp/weights/best.pt --data ../datasets/$DATASET_NAME

classify/val: data=../datasets/Sign-Language-Detectiom-1, weights=['/content/yolov5/runs/train-cls/exp/weights/best.pt'], batch_size=128, imgsz=224, device=, workers=8, verbose=True, project=../content/yolov5/runs/val-cls, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-245-g3d8f004 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 117 layers, 4293507 parameters, 0 gradients, 10.5 GFLOPs
testing: 100% 8/8 [00:01<00:00,  4.04it/s]
                   Class      Images    top1_acc    top5_acc
                     all         997       0.179       0.224
                       1          20           1           1
                       2          15           1           1
                       3          23       0.261           1
                       4          21           1           1
                       5          24           1           1
                       6          12           1           1
                   

### Infer With Your Custom Model

In [8]:
#Get the path of an image from the test or validation set
if os.path.exists(os.path.join(dataset.location, "test")):
  split_path = os.path.join(dataset.location, "test")
else:
  os.path.join(dataset.location, "valid")
example_class = os.listdir(split_path)[0]
example_image_name = os.listdir(os.path.join(split_path, example_class))[0]
example_image_path = os.path.join(split_path, example_class, example_image_name)
os.environ["TEST_IMAGE_PATH"] = example_image_path

print(f"Inferring on an example of the class '{example_class}'")

#Infer
!python /content/yolov5/classify/predict.py --weights /content/yolov5/runs/train-cls/exp/weights/best.pt --source $TEST_IMAGE_PATH

Inferring on an example of the class 'M'
classify/predict: weights=['/content/yolov5/runs/train-cls/exp/weights/best.pt'], source=/datasets/Sign-Language-Detectiom-1/test/M/518_jpg.rf.ff674d0de38ff34ee63eb489650283a5.jpg, data=../content/yolov5/data/coco128.yaml, imgsz=[224, 224], device=, view_img=False, save_txt=False, nosave=False, augment=False, visualize=False, update=False, project=../content/yolov5/runs/predict-cls, name=exp, exist_ok=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-245-g3d8f004 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 117 layers, 4293507 parameters, 0 gradients, 10.5 GFLOPs
image 1/1 /datasets/Sign-Language-Detectiom-1/test/M/518_jpg.rf.ff674d0de38ff34ee63eb489650283a5.jpg: 224x224 M 0.30, L 0.05, N 0.04, Z 0.03, H 0.03, 2.8ms
Speed: 0.3ms pre-process, 2.8ms inference, 0.2ms NMS per image at shape (1, 3, 224, 224)
Results saved to ../content/yolov5/runs/predict-cls/exp


We can see the inference results show ~3ms inference and the respective classes predicted probabilities.

In [9]:
import glob
from IPython.display import Image, display

for imageName in glob.glob('/content/yolov5/runs/detect/exp*.jpg'): #assuming JPG
    display(Image(filename=imageName))
    print("\n")

## (OPTIONAL) Improve Our Model with Active Learning

Now that we've trained our model once, we will want to continue to improve its performance. Improvement is largely dependent on improving our dataset.

We can programmatically upload example failure images back to our custom dataset based on conditions (like seeing an underrpresented class or a low confidence score) using the same `pip` package.

In [12]:

!python /content/yolov5/classify/predict.py --weights /content/yolov5/runs/train-cls/exp/weights/best.pt --source /content/Sign-Language-Detectiom-1/test/C/

classify/predict: weights=['/content/yolov5/runs/train-cls/exp/weights/best.pt'], source=/content/Sign-Language-Detectiom-1/test/C/, data=../content/yolov5/data/coco128.yaml, imgsz=[224, 224], device=, view_img=False, save_txt=False, nosave=False, augment=False, visualize=False, update=False, project=../content/yolov5/runs/predict-cls, name=exp, exist_ok=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-245-g3d8f004 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 117 layers, 4293507 parameters, 0 gradients, 10.5 GFLOPs
image 1/11 /content/Sign-Language-Detectiom-1/test/C/014_jpg.rf.05d55286ecce341e60e8aba3106beb4f.jpg: 224x224 O 0.40, V 0.21, C 0.13, I 0.03, Book 0.02, 4.4ms
image 2/11 /content/Sign-Language-Detectiom-1/test/C/1093_jpg.rf.b835e658765b9c6e24d0d1c26970039c.jpg: 224x224 C 0.21, Y 0.09, T 0.07, I 0.06, O 0.05, 4.6ms
image 3/11 /content/Sign-Language-Detectiom-1/test/C/131_jpg.rf.65da1d9a57649cfd0b8671a5b651a420.jpg: 224

In [ ]:
# # Upload example image
# project.upload(image_path)


In [ ]:
# # Example upload code
# min_conf = float("inf")
# for pred in results:
#     if pred["score"] < min_conf:
#         min_conf = pred["score"]
# if min_conf < 0.4:
#     project.upload(image_path)

# (BONUS) YOLOv5 classify/predict.py Accepts Several Input Methods
- Webcam: `python classify/predict.py --weights yolov5s-cls.pt --source 0`
- Image `python classify/predict.py --weights yolov5s-cls.pt --source img.jpg`
- Video: `python classify/predict.py --weights yolov5s-cls.pt --source vid.mp4`
- Directory: `python classify/predict.py --weights yolov5s-cls.pt --source path/`
- Glob: `python classify/predict.py --weights yolov5s-cls.pt --source 'path/*.jpg'`
- YouTube: `python classify/predict.py --weights yolov5s-cls.pt --source 'https://youtu.be/Zgi9g1ksQHc'`
- RTSP, RTMP, HTTP stream: `python classify/predict.py --weights yolov5s-cls.pt --source 'rtsp://example.com/media.mp4'`

###Directory Example

In [ ]:
#Directory infer
os.environ["TEST_CLASS_PATH"] = test_class_path = os.path.join(*os.environ["TEST_IMAGE_PATH"].split(os.sep)[:-1])
print(f"Infering on all images from the directory {os.environ['TEST_CLASS_PATH']}")
!python classify/predict.py --weights runs/train-cls/exp/weights/best.pt --source /$TEST_CLASS_PATH/

###YouTube Example

In [ ]:
#YouTube infer
!python classify/predict.py --weights runs/train-cls/exp/weights/best.pt --source 'https://www.youtube.com/watch?v=7AlYA4ItA74'